<a href="https://colab.research.google.com/github/acedesci/scanalytics/blob/master/S02_Python_basics/02_Lecture_Examples_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# S2 - Python Basics and Supply Chain Examples - Part I

Programming topics covered in this section:
*    Python functions
*    Condition statements
*    Iteration

Supply chain examples include:
*    EOQ with discounts
*    Bass diffusion model

---
## 1. Python functions

A function is a block to transform [input parameters] -> [output]. We often use a function when a certain operation is used more than once or used in multiple processes. An output can also be returned from a function.

A function usually include the following components:
*   keyword `def`
*   function name
*   parentheses 
*   parameters (variables) or arguments (values) separated by a comma
*   colon
*   code of the function with indentation
*   `return` statement

For example, a function `square` can be written as follows: 
```
def square(x):
    y = x ** 2    
    return y`
```


Calling a function can be done by indicating the function name and provide the required parameters/arguments.

For example, to use the function `square` to calculate the squared value of 10 can be done as follows (this needs to be called strictly after the function is defined and run on Notebook). The following line of code will give the result `squared_value = 100`: 
```
squared_value = square(10)
```

**NOTE:** please also see https://www.w3schools.com/python/python_functions.asp for more descriptions and examples.

In this course, we will make use of function in particular when some operations or processes will be used multiple times or should be organized to avoid clutter.

## `math` Module

The `math` module provides access to several mathematical functions as permutation (`math.perm()`), square root (`math.sqrt()`), rounding  up and rounding down (`math.ceil()` and `math.floor()`, note that the normal rounding function `round()` can be called directly without using module `math`, see [this link](https://www.w3schools.com/python/ref_func_round.asp)).  You can look at [this link](https://docs.python.org/3/library/math.html) for more information about functions in the `math` module. 

In order to use the modules in Python to be able to use the pre-built functions, we can simply `import` the module using the module name prior to using it (typically we import at the beginning of the code), i.e., `import math` then we can call the function `math.sqrt()`.

### Example 1.1: EOQ function
Using the annual demand $D$, the holding cost per unit $H$ and fixed ordering cost $O$, define a function which calculates the economic order quantity (EOQ).

$$ EOQ=\sqrt{\frac{2DO}{H}}$$

Next, call this function to compute the EOQ assuming $D=500$ units per year, $S=\$15$ per order, and $H=\$1.2$ per unit held in inventory. 

**NOTE:** triple quotes `"""..."""` are often used for comments over multiple lines and, in practice, one often adds the descriptions of the parameters, arguments, and the return value of each function. 

In [1]:
# defining the EOQ function

def EOQ(demand, order_cost, holding_cost):
    """
    The text between 
    Compute the economic quantity to order in order to minimize the total costs
    Parameters:
        demand: (number) annual demand
        order_cost: (number) unit ordering cost
        holding_cost: (number different than 0) unit inventory cost
    Return:
        (number) optimal quantity to order
    """
    return ((2 * demand * order_cost) / holding_cost) ** (1/2)

# calculate EOQ using the function
eoq_value=EOQ(demand=600, order_cost=15, holding_cost=1.2)

# another compact way of calling the EOQ function (the sequence must be preserved)
eoq_value = EOQ(600, 15, 1.2)  

print('EOQ = ', eoq_value)

EOQ =  122.47448713915891


### Example 1.2: EOQ rounded up
Define a function which gives the EOQ as integer quantity (rounded up). Use the functions within  `math` module in Python.

In [2]:
# defining EOQ function with rounded number

import math   # importing math module

def ceil_EOQ(demand, order_cost, holding_cost):
    """
    Compute the economic quantity to order as an integer amount
    Parameters:
        demand: (number) annual demand
        order_cost: (number) unit ordering cost
        holding_cost: (number different than 0) unit inventory cost
    Return:
        optimal quantity to order rounded up to the next largest integer
    """
    return  math.ceil(math.sqrt((2 * demand * order_cost) / holding_cost)) 
    # math.ceil(x) is a function to round the number up to the next largest integer.

# calling the Round_EOQ function
ceiling_eoq = ceil_EOQ(demand=600, order_cost=15, holding_cost=1.2)
print('Integer EOQ  = ', ceiling_eoq, ',', type(ceiling_eoq))

Integer EOQ  =  123 , <class 'int'>


---
## 2. Conditional Statements

Conditional expressions are tools for modeling the behavior of a program. The element tested is called a Boolean expression, that is, it can take a true value (`True`) or a false value (`False`). In this expression, several elements can be tested using comparison operations (`==`,`!=`,`>`, `<`,`>=`, `<=`) and Boolean operations (`and`, `or`, `not`). You can look at [this page](https://www.w3schools.com/python/python_conditions.asp) for more information.

**NOTE:** the condition statement is equivalent to IF function in Excel. They are very important statements in programming.

### Example 2.1: EOQ with discounts
EOQ generally minimizes the total inventory cost and ordering cost. However, EOQ may not be optimal when discounts are factored into the calculation. To calculate the EOQ when discounts are involved, we need to perform the following steps:

1. For each quantity range associated with a discount, calculate the quantity to optimal order Q* for that range based on the corresponding parameters using the EOQ equation;
2. **If** Q* does not qualify for a discount, choose the smallest possible order size to get the discount and set Q* to that value, **otherwise** the value Q* remains unchanged;
3. Calculate the total cost for each Q* from step 2;
4. Select the Q* that gives the lowest total cost, which is calculated by. 

$$TC=PD + O\left(\frac{D}{Q}\right) + H\left(\frac{Q}{2}\right)$$

- $TC$: total annual cost;
- $P$: purchase cost per unit;
- $D$: annual demand;
- $O$: fixed cost per order;
- $Q$:  order quantity (quantity $Q^*$ associated with a given discount);
- $H$: unit inventory holding cost = $hP$ where $h$ is the inventory holding cost is given as % of the value of the product.

**Example:** Determine the <u>optimal order quantity</u> given the following information. Demand is forecast as $485$ units per year, the purchase cost is $\$31$ per unit, the inventory holding cost is $\%5$ of the purchase price and the ordering cost is $\$280$. The supplier  is now offering $5\%$ discount on orders above $500$ units.

Please create a function `total_cost` which return the calculation of the total cost based on the parameters provided to the function.

In [3]:
# defining a function total_cost(...) to compute the total costs

def total_cost(demand, acq_cost, order_cost, holding_cost, order_qty):
    """
    Compute the total costs as the sum of the total adquisition cost, ordering cost and inventory cost.
    Parameters:
        demand: (number) annual demand,
        acq_cost: (number) unit acquisition/purchase cost,
        order_cost: (number) fixed ordering cost,
        holding_cost: (number) unit inventory holding cost,
        order_qty: (number) quantity to order.
    Return:
        (number) total costs
    """
    return (acq_cost * demand) + order_cost * (demand / order_qty) + holding_cost * (order_qty/2)

In [4]:
# initialize parameters
D = 485
O = 280
h = 0.05
P = 31

# EOQ 1: determining EOQ and total costs without discounts (discount 0%)
eoq_no_discount = EOQ(demand=D, order_cost=O, holding_cost=h*P)
if eoq_no_discount > 500:
  eoq_no_discount = 500
cost_no_discount = total_cost(demand=D, acq_cost=P, order_cost=O, holding_cost=h*P, order_qty= eoq_no_discount)

print('The order quantity with no discount = ', eoq_no_discount, '; and the corresponding total cost = ', cost_no_discount) 

The order quantity with no discount =  418.59981659290406 ; and the corresponding total cost =  15683.829715719003


In [5]:
# Set the optimal quantity and cost (these values will be updated afterward based on the optimal value)
optimal_order_qty = eoq_no_discount
optimal_total_cost = cost_no_discount

In [6]:
# EOQ 2: calculating the EOQ  for discount 5%
eoq_discount = EOQ(demand=D, order_cost=O, holding_cost=0.95*h*P)
print('The EOQ using the parameters with 5% discount ', eoq_discount)

# checking if 'eoq_discount' does not qualify for the 5% discount and set it eoq_discount to the lower feasible quantity in that case
if eoq_discount <= 500:
  eoq_discount = 501

# computing the total costs with discount 5%
cost_discount = total_cost(demand=D, acq_cost=P*0.95, order_cost=O, holding_cost=0.95*h*P, order_qty= eoq_discount)

print('The order quantity with no discount = ', eoq_discount, '; and the corresponding total cost = ', cost_discount)

The EOQ using the parameters with 5% discount  429.47435001113547
The order quantity with no discount =  501 ; and the corresponding total cost =  14923.169134231537


In [7]:
# choosing the option with a lower cost
if optimal_total_cost > cost_discount:
  optimal_total_cost = cost_discount
  optimal_order_qty = eoq_discount
        
print('The optimal order quantity = ', optimal_order_qty, '; and the corresponding total cost = ', optimal_total_cost)   

The optimal order quantity =  501 ; and the corresponding total cost =  14923.169134231537


---
## 3. `for` and `while` Loops

'for' and 'while' loops are also core programming components which you should effectively leverage. In this context, the loops are used to **iteratively** compute or perform operations. 

You can refer to [this link](https://www.w3schools.com/python/python_for_loops.asp) for the 'for' loops and [this link](https://www.w3schools.com/python/python_while_loops.asp) for the 'while' loops for more examples.

### Example 3.1: Bass diffusion model

Bass diffusion model is one of the most widely adopted models to predict demand (or customer base) over a product life cycle. This model is particularly useful when a new product will be introduced to the market. The background of the model is provided below. 

<blockquote>
  <p> <b>Brief description of the model:</b> In this model, the cumulative number of adoptors (people who bought/adopted the product) $S(t)$ from the beginning (time 0) to time $t$ is described by $S(t) = m\times F(t)$ where $m$ is the market size (the estimated total number of potential customers) and $F(t)$ is the <i>cumulative</i> probability that an individual (potential customer) has already adopted the product by time $t$ (which needs to be calculated). In other words, $F(t)$ represents the portion of potential market $m$ that have adopted by time t. Thus, the value $1 - F(t)$ represents the portion of $m$ that have <i>not yet</i> adopted the product by time $t$.
<br /> 
Consequently, we can also write $f(t) = \frac{d}{dt}F(t)$ where $f(t)$ is the probability that an individual adopts the product at time $t$. 
<br /> 
Bass's model, proposed by Frank Bass, is described as follows:
<br /> 
$$\frac{f(t)}{1-F(t)} = \frac{\frac{d}{dt}F(t)}{1-F(t)} = p + qF(t)$$
where 
<br /> 
* $p$ is the coefficiant of innovation
<br /> 
* $q$ is the coefficient of imitation
<br /> 
which indicates that the rate of adoptions for those who have not adopted prior to time $t$ is equal to $p+qF(t)$. One can observe that $p$ is *not* associated with the overall market status (which is captured by $F(t)$) and represents the the rate of adoptions from an individual which is *independent* of the others, whereas $q$ is associated with the market status $F(t)$ and represents the adoption influenced by other adoptors (by imitation). These values $p$ and $q$ are used by marketers/planners to describe the behavior of the customer base for their new product. 
<br /> 
<i><b>Note:</b> In practice, values of $p$ and $q$ are estimated using a regression technique applied to the *similar* products already sold in the market (where data is already available). In this example, we assume that $m$, $p$ and $q$ are already available for the calculation. We will revisit the parameter estimation ($p$, $q$ and $m$) of this model from sales data in a later session.</i>
<br /> 
Based on the derivative of the Bass diffussion function above (detail omitted here), we can obtain $F(t)$ as a function of $p$ and $q$ as follows:

$$F(t) = \frac{1-e^{-(p+q)t}}{1+\frac{q}{p}e^{-(p+q)t}} $$

Consequently, the total cumulative adoptors (demand) $S(t)$ up to $t$ can be calculated as $S(t) = m\times F(t)$. More details on the Bass diffusion model can be found on [this link](https://srdas.github.io/MLBook/productForecastingBassModel.html) and [this link](https://en.wikipedia.org/wiki/Bass_diffusion_model).
<br /> 
<b>Reference</b>: Bass, F. M. (1969). A new product growth for model consumer durables. Management science, 15(5), 215-227.
</p> 
</blockquote>


Given a value of $m$, $p$ and $q$, we want to determine the number of adoptions using the Bass model above for years 1 to 5 for the following cases:

* Product 1: $m_1=1000$, $q_1 = 0.40$, and $p_1 = 0.10$
* Product 2: $m_2=1000$, $q_2 = 0.10$, and $p_2 = 0.40$

First, we determine the function to calculate $F(t)$

In [8]:
import math
def Bass_cumulative_probability_Ft(p, q, t):
    return (1-math.exp(-(p+q)*t))/(1+(q/p)*math.exp(-(p+q)*t))


Next, we perform the calculations and print out the number of adoptions for each year using `for` loop:



In [9]:
m_product_1 = 1000
q_product_1 = 0.4
p_product_1 = 0.1


print("Cumulative adoptions (demand) prediction for product 1")
# Note that [] defines a list which contains ordered values. 
# We will discuss in more detail the data structure in the next session
for t in [1,2,3,4,5]:
  Ft_product_1 = Bass_cumulative_probability_Ft(p_product_1, q_product_1, t)
  print("Year[",t,"]: F(t) = ", Ft_product_1,", Cumulative N. of adoptions until t = ", m_product_1*Ft_product_1)


Cumulative adoptions (demand) prediction for product 1
Year[ 1 ]: F(t) =  0.1148439159257229 , Cumulative N. of adoptions until t =  114.84391592572291
Year[ 2 ]: F(t) =  0.2557620939896121 , Cumulative N. of adoptions until t =  255.76209398961208
Year[ 3 ]: F(t) =  0.4104947778048284 , Cumulative N. of adoptions until t =  410.4947778048284
Year[ 4 ]: F(t) =  0.5609820553549241 , Cumulative N. of adoptions until t =  560.9820553549241
Year[ 5 ]: F(t) =  0.6910241392864611 , Cumulative N. of adoptions until t =  691.0241392864611


In [10]:
m_product_2 = 1000
q_product_2 = 0.1
p_product_2 = 0.4

print("Cumulative adoptions (demand) prediction for product 2")
for t in range(1,6):
  Ft_product_2 = Bass_cumulative_probability_Ft(p_product_2, q_product_2, t)
  print("Year[",t,"]: f(t) = ", Ft_product_2,", Cumulative N. of adoptions until t = ", m_product_2*Ft_product_2)


Cumulative adoptions (demand) prediction for product 2
Year[ 1 ]: f(t) =  0.3416621916606648 , Cumulative N. of adoptions until t =  341.6621916606648
Year[ 2 ]: f(t) =  0.578880957995513 , Cumulative N. of adoptions until t =  578.880957995513
Year[ 3 ]: f(t) =  0.7358237235333194 , Cumulative N. of adoptions until t =  735.8237235333194
Year[ 4 ]: f(t) =  0.8363672181730698 , Cumulative N. of adoptions until t =  836.3672181730698
Year[ 5 ]: f(t) =  0.8994570193276593 , Cumulative N. of adoptions until t =  899.4570193276593


### Example 3.2: Bass diffusion using `while` loop

`while` loop must be defined with a condition and the loop continues until the condition is no longer met.
 
Repeat Exercise 3.1 using a `while` loop.

In [11]:
t = 1 # set the initial value of t
print("Cumulative adoptions (demand) prediction for product 1")
while t <= 5:
  Ft_product_1 = Bass_cumulative_probability_Ft(p_product_1, q_product_1, t)
  print("Year[",t,"]: F(t) = ", Ft_product_1,", Cumulative N. of adoptions until t = ", m_product_1*Ft_product_1)
  t = t + 1

Cumulative adoptions (demand) prediction for product 1
Year[ 1 ]: F(t) =  0.1148439159257229 , Cumulative N. of adoptions until t =  114.84391592572291
Year[ 2 ]: F(t) =  0.2557620939896121 , Cumulative N. of adoptions until t =  255.76209398961208
Year[ 3 ]: F(t) =  0.4104947778048284 , Cumulative N. of adoptions until t =  410.4947778048284
Year[ 4 ]: F(t) =  0.5609820553549241 , Cumulative N. of adoptions until t =  560.9820553549241
Year[ 5 ]: F(t) =  0.6910241392864611 , Cumulative N. of adoptions until t =  691.0241392864611


In [12]:
t = 1 # set the initial value of t
print("Cumulative adoptions (demand) prediction for product 2")
while t <= 5:
  Ft_product_2 = Bass_cumulative_probability_Ft(p_product_2, q_product_2, t)
  print("Year[",t,"]: f(t) = ", Ft_product_2,", Cumulative N. of adoptions until t = ", m_product_2*Ft_product_2)
  t += 1 # this is equivalent to t = t + 1

Cumulative adoptions (demand) prediction for product 2
Year[ 1 ]: f(t) =  0.3416621916606648 , Cumulative N. of adoptions until t =  341.6621916606648
Year[ 2 ]: f(t) =  0.578880957995513 , Cumulative N. of adoptions until t =  578.880957995513
Year[ 3 ]: f(t) =  0.7358237235333194 , Cumulative N. of adoptions until t =  735.8237235333194
Year[ 4 ]: f(t) =  0.8363672181730698 , Cumulative N. of adoptions until t =  836.3672181730698
Year[ 5 ]: f(t) =  0.8994570193276593 , Cumulative N. of adoptions until t =  899.4570193276593
